In [1]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
from tqdm import tqdm
import numpy as np

In [2]:
model = SentenceTransformer("Santarabantoosoo/songs_fine-tuned-all-MiniLM-L12-v2")

c:\Users\berzi\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [3]:
df = pd.read_csv("data/spotify_millsongdata_w_genres.csv")

In [4]:
song_embeddings = model.encode(df['song'].values, show_progress_bar=True)
lyrics_embeddings = model.encode(df['cleaned_text'].values, show_progress_bar=True)
genres_embeddings = model.encode(df['genres'].values, show_progress_bar=True)

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

In [5]:
embeddings = []
for i in range(len(song_embeddings)):
    embeddings.append((song_embeddings[i] + lyrics_embeddings[i] + genres_embeddings[i]) / 3)

In [6]:
cos_sim = util.cos_sim(embeddings, embeddings)

c:\Users\berzi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sentence_transformers\util.py:38: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:248.)
  a = torch.tensor(a)


In [7]:
cos_sim.shape

torch.Size([100, 100])

In [27]:
song_index = 50

In [28]:
similar_songs = np.flip(np.argsort(cos_sim[song_index].numpy()))[1:10]

In [29]:
df.loc[song_index]

artist                                                       ABBA
song                                      Lay All Your Love On Me
cleaned_text    i was n't jealous before we met now every woma...
genres                                                   Rock,Pop
Name: 50, dtype: object

In [30]:
odf = df.loc[similar_songs]
odf['similarity'] = cos_sim[song_index].numpy()[similar_songs]
odf

,artist,song,cleaned_text,genres,similarity
44,ABBA,I've Been Waiting For You,"i , i 've been in love before i thought i woul...","Rock,Pop",0.763489
55,ABBA,Lovers,sit down and listen 'cause i 've got good news...,"Pop,Rock",0.741409
53,ABBA,Love Isn't Easy,"do you remember the first time , and all of yo...","Pop,Rock",0.740054
52,ABBA,Love Has It's Ways,spring in the air the sun was shining still th...,"Rock,Pop",0.724492
64,ABBA,"My Love, My Life",i 've seen it on your face tells me more than ...,"Rock,Pop",0.720051
77,ABBA,Rock Me,"rock me , give me that kick now rock me , show...","Rock,Pop",0.718103
34,ABBA,I Am Just A Girl,"i am just a girl one among the others , nothin...","Rock,Pop",0.717480
92,ABBA,That's Me,are you sure you want to hear more what if i a...,"Rock,Pop",0.714549
90,ABBA,Take A Chance On Me,"if you change your mind , i 'm the first in li...","Pop,Rock",0.713849
